In [1]:
import numpy as np
import pandas as pd
d0 = pd.to_datetime('2021-9-18')  #Last Coupon Date
d1 = pd.to_datetime('2021-11-3')  #Pricing Date
d2 = pd.to_datetime('2022-9-18')  #Next Coupon Date
dt=(d2-d1).days  #Days of Next Coupon Date minus Pricing Date
TS0=(d2-d0).days  #Days of Next Coupon Date minus Last Coupon Date

#Full Price
'''Function of Full Price
    y:YTM
    C:Annual Coupon Payment；
    f:Coupon Frequency；
    d:dt；
    n:Times of coupon payments bewteen Pricing Date and Maturity；
    M:Par Value;
    TS:Days of Next Coupon Date minus Last Coupon Date。'''
def Bond_price(y,C,f,d,n,M,TS):
    import numpy as np
    coupon=[]
    for i in np.arange(0,n):
       coupon.append((C/f)/pow(1+y/f,d/TS+i))
    return np.sum(coupon)+M/pow(1+y/f,d/TS+n-1)

Bondprice=Bond_price(y=0.02569029,C=6.875,f=1,d=dt,n=2,M=100,TS=TS0)
print('PV of Full Price',np.round(Bondprice,4))

PV of Full Price 108.6378


In [3]:
y=0.02569029 
C=6.875 
f=1    
d=dt  
n=2   
M=100  
TS=TS0 
#d0 = pd.to_datetime('2019-11-22') 
#d1 = pd.to_datetime('2020-1-10') 

#Generate Discount Flow
import numpy as np
cashflow=[]
discountcashflow=[]
for i in np.arange(0,n):
    discountcashflow.append((C/f)/pow(1+y/f,d/TS+i))
    cashflow.append(C/f)       
discountcashflow[n-1]=discountcashflow[n-1]+M/pow(1+y/f,d/TS+n-1)
cashflow[n-1]=cashflow[n-1]+M

#Cash Flow Date
import datetime
import calendar
import math
def get_recent_month(dt, months):
    month = dt.month - 1 + months
    year = math.floor(dt.year + month / 12)
    month = month % 12 + 1
    day = min(dt.day, calendar.monthrange(year, month)[1])
    return str(dt.replace(year=year, month=month, day=day))[:10]

#Compute All Next Coupon Date
period_list = []
date=[]
for j in range(1,n+1):
    j=j*int(12/f)+1
    for i in range(1, j):
      period_list.append(get_recent_month(d0, i))
    date.append(get_recent_month(d0, j-1))
date=pd.to_datetime(date)
print (date)

#Annual Time bewteen Current and Next Coupon Date
t1 = [d] 
for i in range(len(date)-1):
    b = (date[i+1] - date[i]).days
    t1.append(b)

#Are next few years are leap years?
t1[0]=t1[0]/366.0    #If next coupon date is leap year
for i in range(1,n):
     if date[i].year%4==0:
         t1[i]=t1[i]/366
     else:
         t1[i]=t1[i]/365
     t1[i]=t1[i-1]+t1[i]

#Generate the table
frame1=pd.DataFrame({'date':date,'year':t1,'cashflow':cashflow,
                     'discountcashflow':discountcashflow,
                     'weight':discountcashflow/Bondprice,
                     'weight_mul_year':(np.array(t1)*discountcashflow)/Bondprice,
                     })  
new=pd.DataFrame({'date':d1,
         'year':0,
         'cashflow':0,
         'discountcashflow':0,
         'weight':0,
         'weight_mul_year':0,},index=[0])
frame1=new.append(frame1,ignore_index=True)   #Date

new2=pd.DataFrame({'date':d1,
         'year':'summation',
         'cashflow':sum(frame1.cashflow),
         'discountcashflow':sum(frame1.discountcashflow),
         'weight':sum(frame1.weight),
         'weight_mul_year':sum(frame1.weight_mul_year)},index=[0])
frame2=frame1.append(new2,ignore_index=True)   #Date
pd.set_option('display.width', 1000)     #Width of rows
pd.set_option('display.max_rows', None)
print(frame2)

DatetimeIndex(['2022-09-18', '2023-09-18'], dtype='datetime64[ns]', freq=None)
        date       year  cashflow  discountcashflow    weight  weight_mul_year
0 2021-11-03        0.0     0.000          0.000000  0.000000         0.000000
1 2022-09-18   0.871585     6.875          6.724265  0.061896         0.053948
2 2023-09-18   1.871585   106.875        101.913564  0.938104         1.755741
3 2021-11-03  summation   113.750        108.637828  1.000000         1.809689
